In [1]:
import pandas as pd
import datetime as dt

from markets_analytics import redshift, exasol, datalake, etl, dateutil, GSheetHelper

# 1. Database Connectors

## Redshift Connector

In [2]:
schema = 'sales_and_supply'
table = 'library_test'

In [3]:
redshift.test_connection()

Username `sales_and_supply_user` connected to Redshift host: lounge-dwh-production.cotnm1vpt3gw.eu-central-1.redshift.amazonaws.com:5439


In [4]:
query = '''
    DROP TABLE IF EXISTS {schema}.{table};
    
    CREATE TABLE IF NOT EXISTS {schema}.{table} (
        name VARCHAR(10),
        age INT
    );
'''.format(schema=schema, table=table)

redshift.create(query)

[2024-07-10 16:52:44.553393] Creating Table
[2024-07-10 16:52:44.929802] Table Created Successfully


In [5]:
df = pd.DataFrame({
    'name': ['X', 'Y', 'Z'],
    'age': [31, 29, 20]
})

redshift.insert(df, schema, table)

query = '''
    SELECT * FROM {}.{}
'''.format(schema, table)

redshift.execute(query)

[2024-07-10 16:52:44.936306] Inserting data to sales_and_supply.library_test
[2024-07-10 16:52:45.754446] Data Inserted Successfully
[2024-07-10 16:52:45.754792] Executing Query
[2024-07-10 16:52:45.837780] Query Executed Successfully


,name,age
0,X,31
1,Y,29
2,Z,20


In [6]:
redshift.delete(schema, table, 'age=31')

query = '''
    SELECT * FROM {}.{}
'''.format(schema, table)

redshift.execute(query)

[2024-07-10 16:52:45.852042] Deleting data from sales_and_supply.library_test
[2024-07-10 16:52:46.560052] Deleted Data Successfully
[2024-07-10 16:52:46.560411] Executing Query
[2024-07-10 16:52:46.647078] Query Executed Successfully


,name,age
0,Y,29
1,Z,20


In [7]:
values = "name='A'"
redshift.update(schema, table, values, "age=20 AND name='Z'")

query = '''
    SELECT * FROM {}.{}
'''.format(schema, table)

redshift.execute(query)

[2024-07-10 16:52:46.655101] Updating data for sales_and_supply.library_test
[2024-07-10 16:52:47.058560] Updated Data Successfully
[2024-07-10 16:52:47.058850] Executing Query
[2024-07-10 16:52:47.136448] Query Executed Successfully


,name,age
0,Y,29
1,A,20


## Datalake Connector

In [8]:
datalake.test_connection()

Username `dimran` connected to Datalake host: interactive.starburst.zalando.net:443


In [9]:
query = '''
    SELECT
        *
    FROM dwh_insider.f_salesorder_position
    LIMIT 5
'''

datalake.execute(query)

[2024-07-10 16:52:47.241211] Executing Query
[2024-07-10 16:52:49.056775] Query Executed Successfully


,sk_order_date,order_position_number,order_date,order_year,order_rank_overall,partner_flag,flag_coupon_used,flag_cancelled_order,cancellation_date,financial_date,...,flag_delivery_is_missing,cancellation_status,locale,commission_type,fee_category,partner_commission_revenue_aret,net_zfs_incentive_aret,net_sus_incentive_aret,net_lch_incentive_aret,net_aut_incentive_aret
0,20090107,1.0,2009-01-07,2009.0,3.0,0.0,0.0,1.0,2012-05-06,None,...,None,None,None,None,None,None,None,None,None,None
1,20081002,1.0,2008-10-02,2008.0,1.0,0.0,0.0,0.0,NaT,None,...,None,None,None,None,None,None,None,None,None,None
2,20081011,2.0,2008-10-11,2008.0,1.0,0.0,0.0,0.0,NaT,None,...,None,None,None,None,None,None,None,None,None,None
3,20080930,1.0,2008-09-30,2008.0,1.0,0.0,0.0,1.0,2012-05-06,None,...,None,None,None,None,None,None,None,None,None,None
4,20081002,1.0,2008-10-02,2008.0,1.0,0.0,0.0,0.0,NaT,None,...,None,None,None,None,None,None,None,None,None,None


## Exasol Connector

In [10]:
exasol.test_connection()

Username `ROBOT_OFFPRICE_SALES` connected to Exasol host: bi-prod.exasol.zalando.net:8563


In [11]:
query = '''
    SELECT
        *
    FROM ZALANDO_LOUNGE_ANALYSE.D_CUSTOMER_LOUNGE
    LIMIT 3
'''

exasol.execute(query)

[2024-07-10 16:52:49.208848] Executing Query
[2024-07-10 16:52:50.952195] Query Executed Successfully


,SK_CUSTOMER,CUSTOMER_ORIGIN,LAST_ORDER_DATE_LOUNGE,TOTAL_ORDERS_SHOP,TOTAL_ORDERS_LOUNGE,LOGIN_ALL_DEVICES_LAST_7_DAYS,LOGIN_ALL_DEVICES_LAST_30_DAYS,LOGIN_ALL_DEVICES_LAST_90_DAYS,SIGNUP_DATE,SIGNUP_COUNTRY,...,CUSTOMER_VALUE_SEGMENT,IS_LOUNGE_SIGNUP,LOUNGE_SIGNUP_TIME,IS_TEST_USER,CREATED_AT,LAST_MODIFIED,EXA_INSERTED_AT,IS_DELETED_USER,REGISTRATION_COUNTRY,CUSTOMER_GENDER
0,228756625,SHOP,NaN,3.0,0.0,0,0,0,NaN,NaN,...,Passive D,0,NaN,0,2023-11-07 15:55:15.672000,2024-05-24 06:27:31.661000,2024-05-26 08:26:10.302000,1,FR,OTHER
1,202979079,SHOP,NaN,12.0,0.0,0,0,0,NaN,NaN,...,Passive C,0,NaN,0,2023-11-07 15:55:15.672000,2024-03-11 11:16:19.733000,2024-03-13 10:26:03.116000,0,NO,FEMALE
2,1190647287,SHOP,NaN,NaN,NaN,0,0,0,NaN,NaN,...,NaN,0,NaN,0,2023-11-07 15:55:15.672000,2023-11-07 15:55:15.672000,2023-11-09 10:20:02.084000,0,AT,MALE


# 2. ETL Pipelines

## Pipeline Status

In [12]:
etl.get_source_count('reporting', 'f_lounge_sales_order_position', 'order_date')

177651

In [13]:
etl.get_source_status('reporting', 'f_lounge_sales_order_position', 'order_date', 50000)

True

In [14]:
etl.get_pipeline_status('customer_orders_daily')

'Not Executed Yet'

In [15]:
etl.get_pipeline_runs('customer_orders_daily')

0

## Pipeline Design

In [42]:
schema = 'sales_and_supply'
table = 'pipeline_example'

query = '''
    DROP TABLE IF EXISTS {schema}.{table};
    
    CREATE TABLE IF NOT EXISTS {schema}.{table} (
        name VARCHAR(16),
        team VARCHAR(32)
    );
'''.format(schema=schema, table=table)

redshift.create(query, log=False)

df = pd.DataFrame({
    'name': ['Viktor', 'Franzi', 'Gosia', 'Kashyap', 'Mounav', 'Bruno'],
    'team': ['Markets Analytics', 'Market Research', 'Demand PSPM', 'Markets Analytics', 'Pricing', 'CP&S']
})

redshift.insert(df, schema, table, log=False)

query = 'SELECT * FROM {}.{}'.format(schema, table)

redshift.execute(query, log=False)

,name,team
0,Viktor,Markets Analytics
1,Franzi,Market Research
2,Gosia,Demand PSPM
3,Kashyap,Markets Analytics
4,Mounav,Pricing
5,Bruno,CP&S


In [43]:
# with exception
import random 

try:
    is_src_updated = etl.get_source_status('reporting', 'd_customer_lounge', 'created_at', threshold=10000)
    is_pipeline_exec = etl.get_pipeline_runs('pipeline_example') >= 1
    
    etl.start_pipeline(table)
    
    raise Exception()

    print('Is Source Updated: {}\nDid Pipeline Execute?: {}'.format(is_src_updated, is_pipeline_exec))

    names = ['Minh', 'Ghada', 'Patrick', 'Danyal', 'Tamara', 'Eugenia', 'Tommi', 'Nick']
    department = ['Markets Analytics', 'Demand', 'Pricing', 'Supply', 'PMO', 'Market Research', 'Finance', 'LMT']

    if is_src_updated and not is_pipeline_exec:
        rand = random.randint(0, 7)

        df = pd.DataFrame({
            'name': [names[rand]],
            'team': [department[rand]]
        })

        redshift.insert(df, schema, table, log=False)
        etl.end_pipeline(table)
except:
    etl.delete_run(table)
    
redshift.execute('SELECT * FROM sales_and_supply.pipeline_status', log=False)

,process_name,start_time,end_time,status,message


In [44]:
# without exception
import random 

try:
    is_src_updated = etl.get_source_status('reporting', 'd_customer_lounge', 'created_at', threshold=10000)
    is_pipeline_exec = etl.get_pipeline_runs('pipeline_example') >= 1
    
    etl.start_pipeline(table)

    print('Is Source Updated: {}\nDid Pipeline Execute?: {}'.format(is_src_updated, is_pipeline_exec))

    names = ['Minh', 'Ghada', 'Patrick', 'Danyal', 'Tamara', 'Eugenia', 'Tommi', 'Nick']
    department = ['Markets Analytics', 'Demand', 'Pricing', 'Supply', 'PMO', 'Market Research', 'Finance', 'LMT']

    if is_src_updated and not is_pipeline_exec:
        rand = random.randint(0, 7)

        df = pd.DataFrame({
            'name': [names[rand]],
            'team': [department[rand]]
        })

        redshift.insert(df, schema, table, log=False)
        etl.end_pipeline(table)
except:
    etl.delete_run(table)
    
redshift.execute('SELECT * FROM sales_and_supply.pipeline_status', log=False)

Is Source Updated: True
Did Pipeline Execute?: False


,process_name,start_time,end_time,status,message
0,pipeline_example,2024-07-10 16:55:56.945528,2024-07-10 16:55:58.943156,Completed,


In [45]:
query = 'SELECT * FROM {}.{}'.format(schema, table)

redshift.execute(query, log=False)

,name,team
0,Viktor,Markets Analytics
1,Franzi,Market Research
2,Gosia,Demand PSPM
3,Kashyap,Markets Analytics
4,Mounav,Pricing
5,Bruno,CP&S
6,Ghada,Demand


# 3. Google Sheets

In [20]:
gsheet = GSheetHelper('Intra-Week Tool v2.2')

## Read

In [21]:
gsheet.read('Report - CW', 'I4')

[2024-07-10 16:53:04.991914] Reading from Tab: Report - CW


'The tool did not update at all since 10.07.2023'

In [22]:
gsheet.read('Report - CW', 'I275:Q283', False)

[2024-07-10 16:53:05.412147] Reading from Tab: Report - CW


,0,1,2,3,4,5,6,7,8
0,Orders bCanc,CW Act. + Cherry + FC,"72,335","72,393","68,919","90,203","105,017","107,610","111,847"
1,,vs FC %,-13.54%,-12.32%,-20.45%,0.00%,0.00%,0.00%,0.00%
2,,Forecast,"83,667","82,565","86,633","90,203","105,017","107,610","111,847"
3,,vs Case %,-18.93%,-16.83%,-25.36%,-15.01%,-1.65%,4.75%,-0.76%
4,,Case,"89,224","87,046","92,330","106,131","106,781","102,729","112,706"
5,,vs YoY %,-16.08%,-9.82%,-17.15%,-10.85%,30.61%,39.84%,40.98%
6,,LY - Actuals,"86,194","80,277","83,181","101,176","80,406","76,954","79,333"
7,,vs WoW %,-21.03%,-25.19%,-29.87%,6.29%,26.03%,47.38%,39.16%
8,,LW - Actuals,"91,600","96,775","98,276","84,866","83,327","73,015","80,376"


In [23]:
gsheet.read('Report - CW', 'I14:Q22')

[2024-07-10 16:53:05.849672] Reading from Tab: Report - CW


,GMV bCanc eSC (M),CW Act. + Cherry + FC,€6.127,€6.258,€5.904,€7.947,€8.904,€9.099,€9.533
0,,vs FC %,-17.16%,-15.28%,-23.36%,0.00%,0.00%,0.00%,0.00%
1,,Forecast,€7.396,€7.387,€7.703,€7.947,€8.904,€9.099,€9.533
2,,vs Case %,-19.72%,-16.88%,-26.64%,-13.79%,-4.50%,1.03%,-2.71%
3,,Case,€7.632,€7.530,€8.048,€9.218,€9.323,€9.006,€9.799
4,,vs YoY %,-6.12%,-3.33%,-11.13%,2.44%,39.53%,21.03%,32.11%
5,,LY - Actuals,€6.527,€6.474,€6.643,€7.757,€6.381,€7.518,€7.216
6,,vs WoW %,-22.52%,-27.04%,-30.67%,7.82%,23.46%,44.74%,21.88%
7,,LW - Actuals,€7.908,€8.577,€8.516,€7.370,€7.212,€6.286,€7.822


In [24]:
gsheet.read('Report - CW', 'I14:Q22', header=True, header_range='I7:Q7')

[2024-07-10 16:53:06.330314] Reading from Tab: Report - CW


,,,2024-07-08,2024-07-09,2024-07-10,2024-07-11,2024-07-12,2024-07-13,2024-07-14
0,GMV bCanc eSC (M),CW Act. + Cherry + FC,€6.127,€6.258,€5.904,€7.947,€8.904,€9.099,€9.533
1,,vs FC %,-17.16%,-15.28%,-23.36%,0.00%,0.00%,0.00%,0.00%
2,,Forecast,€7.396,€7.387,€7.703,€7.947,€8.904,€9.099,€9.533
3,,vs Case %,-19.72%,-16.88%,-26.64%,-13.79%,-4.50%,1.03%,-2.71%
4,,Case,€7.632,€7.530,€8.048,€9.218,€9.323,€9.006,€9.799
5,,vs YoY %,-6.12%,-3.33%,-11.13%,2.44%,39.53%,21.03%,32.11%
6,,LY - Actuals,€6.527,€6.474,€6.643,€7.757,€6.381,€7.518,€7.216
7,,vs WoW %,-22.52%,-27.04%,-30.67%,7.82%,23.46%,44.74%,21.88%
8,,LW - Actuals,€7.908,€8.577,€8.516,€7.370,€7.212,€6.286,€7.822


## Write

In [25]:
gsheet.write('Report - CW', 'The tool did not update at all since 10.07.2023', 'I4')

[2024-07-10 16:53:07.173161] Writing to Tab: Report - CW
[2024-07-10 16:53:07.789003] Data Written Successful


In [26]:
df = pd.DataFrame({
    'name': ['X', 'Y', 'Z'],
    'age': [31, 29, 20]
})

gsheet.write('Test', df, 'B2')

[2024-07-10 16:53:07.795002] Writing to Tab: Test
[2024-07-10 16:53:08.374932] Data Written Successful


## Delete

In [27]:
gsheet.delete('Test', 'B2:D4')

[2024-07-10 16:53:08.380708] Delete from Tab: Test
[2024-07-10 16:53:09.970369] Data Deleted Successful


# 4. Date Utility

## Date Ranges

In [28]:
print(dateutil.get_isoweek_range(28))
print(dateutil.get_isoweek_range(1, 2023))

(datetime.date(2024, 7, 8), datetime.date(2024, 7, 14))
(datetime.date(2022, 12, 26), datetime.date(2023, 1, 1))


In [29]:
date = '2024-05-09'
print(dateutil.get_start_of_timeframe(date, 'week'))
print(dateutil.get_start_of_timeframe(date, 'month'))
print(dateutil.get_start_of_timeframe(date, 'quarter'))
print(dateutil.get_start_of_timeframe(date, 'year'))

2024-05-06
2024-05-01
2024-04-01
2024-01-01


In [30]:
date = '2014-12-30'
print(dateutil.get_end_of_timeframe(date, 'week'))
print(dateutil.get_end_of_timeframe(date, 'month'))
print(dateutil.get_end_of_timeframe(date, 'quarter'))
print(dateutil.get_end_of_timeframe(date, 'year'))

2015-01-04
2014-12-31
2014-12-31
2014-12-31


In [31]:
date = '2020-01-02'
print(dateutil.get_start_and_end_of_timeframe(date, 'week'))
print(dateutil.get_start_and_end_of_timeframe(date, 'month'))
print(dateutil.get_start_and_end_of_timeframe(date, 'quarter'))
print(dateutil.get_start_and_end_of_timeframe(date, 'year'))

(datetime.date(2019, 12, 30), datetime.date(2020, 1, 5))
(datetime.date(2020, 1, 1), datetime.date(2021, 1, 31))
(datetime.date(2020, 1, 1), datetime.date(2020, 3, 31))
(datetime.date(2020, 1, 1), datetime.date(2020, 12, 31))


## Till Date and Outlook

In [32]:
query = '''
    SELECT
        order_date,
        EXTRACT(WEEKDAY FROM order_date) AS weekday,
        SUM(gmv_bef_cancellation) AS gmv
    FROM reporting.f_lounge_sales_order_position
    WHERE order_date BETWEEN '2024-01-01' AND '2024-01-31'
    GROUP BY order_date
    ORDER BY order_date
'''

df = redshift.execute(query)
df

[2024-07-10 16:53:10.017000] Executing Query
[2024-07-10 16:53:10.090906] Query Executed Successfully


,order_date,weekday,gmv
0,2024-01-01,1,7.758236e+06
1,2024-01-02,2,8.606549e+06
2,2024-01-03,3,8.683049e+06
3,2024-01-04,4,9.756538e+06
4,2024-01-05,5,9.236966e+06
5,2024-01-06,6,1.183327e+07
6,2024-01-07,0,1.187348e+07
7,2024-01-08,1,9.264072e+06
8,2024-01-09,2,1.013480e+07
9,2024-01-10,3,9.443912e+06


In [33]:
dateutil.get_till_date_df(df, 'order_date', date_today='2024-01-17')

,order_date,weekday,gmv
14,2024-01-15,1,8.828302e+06
15,2024-01-16,2,8.822254e+06
16,2024-01-17,3,9.554627e+06


In [34]:
dateutil.get_outlook_df(df, 'order_date', date_today='2024-01-17')

,order_date,weekday,gmv
14,2024-01-15,1,8.828302e+06
15,2024-01-16,2,8.822254e+06
16,2024-01-17,3,9.554627e+06
17,2024-01-18,4,9.056814e+06
18,2024-01-19,5,8.214310e+06
19,2024-01-20,6,1.038267e+07
20,2024-01-21,0,9.782504e+06


In [35]:
date = '2024-01-08'
cw_range = dateutil.get_start_and_end_of_timeframe(date, 'week')
cw_df = df[(df['order_date'] >= cw_range[0]) & (df['order_date'] <= cw_range[1])]
cw_td_df = df[(df['order_date'] >= cw_range[0]) & (df['order_date'] <= pd.to_datetime(date).date())]

print('GMV WTD: €{:,.0f}'.format(cw_td_df['gmv'].sum()))
print('GMV Outlook: €{:,.0f}'.format(cw_df['gmv'].sum()))

GMV WTD: €9,264,072
GMV Outlook: €72,912,927
